# Analysis

In [56]:
import time
import tqdm
import pandas as pd
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt
import seaborn as sns
import re
import lemmy # For lemmatization
import nltk
from nltk.stem import SnowballStemmer
import itertools

from sklearn.feature_extraction.text import CountVectorizer


## Load datasets

In [34]:
ft_sygeplej2x = pd.read_csv('ft_sygeplej2x.csv')
dr_sygeplej2x = pd.read_csv('dr_sygeplej2x.csv')
tv2_sygeplej2x = pd.read_csv('tv2_sygeplej2x.csv')

ft_2 = ft_sygeplej2x.copy() 
dr_2 = dr_sygeplej2x.copy() 
tv2_2 = tv2_sygeplej2x.copy()

# Preprocessing

## Remove non-alphanumerical characters

In [3]:
for df in [ft_2, dr_2, tv2_2]:
    df['content'] = df['content'].str.replace(r'\W', ' ')\
                                 .str.replace('  ', ' ')

/var/folders/g1/m2905s6d6093hshyj_v9xm380000gn/T/ipykernel_23493/1145022020.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  df['content'] = df['content'].str.replace(r'\W', ' ')\


## Tokenization

In [4]:
# Download tokenizer
nltk.download("punkt")

[nltk_data] Downloading package punkt to /Users/oliver/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [36]:
dr_2['tokenized'] = df.apply(lambda row: nltk.tokenize.word_tokenize(row['content']), axis=1)

In [37]:
 stemmed_tok = [porter_stemmer.stem(token) for token in tokens]

0      [mens, nyuddannede, danske, sygeplejersker, ha...
1      [antallet, af, danske, sygeplejersker, der, ha...
2      [en, bevæbnet, mand, har, skudt, og, dræbt, sy...
3      [hvert, år, dør, hundredvis, af, patienter, fo...
4      [sygeplejersker, lærere, politifolk, og, unge,...
                             ...                        
523    [sundhedsstyrelsen, kom, torsdag, med, hård, k...
524    [stop, det, offentlige, forbrug, og, sænk, ska...
525    [der, er, mange, forsinkelser, og, fejl, i, va...
526    [36, årige, kirsten, brosbøl, bliver, ny, milj...
527    [sundhedsmyndighederne, i, usa, har, mandag, a...
Name: tokenized, Length: 528, dtype: object

In [49]:
dr_2['stemmed'] = df.apply(lambda row: [stemmer.stem(['tokenized']) for word in ['tokenized']], axis=1)



AttributeError: 'list' object has no attribute 'lower'

In [50]:
def stem_sentences(sentence):
    tokens = sentence.split()
    stemmed_tokens=[stemmer.stem(word) for word in tokens]
    no
    stems=' '.join(stemmed_tokens)
    return stems


In [53]:
stemmed_list=[]
for i in dr_2['content']:
    stems=stem_sentences(i)
    stemmed_list.append(stems)

dr_2['stems']=stemmed_list

In [55]:
dr_2

,Unnamed: 0,titles,sub_header,h2,content,author,tag,date,link,source,tokenized,stems
0,0,danske sygeplejersker får job i norge,"krise, fyringer og ansættelsesstop får sygeple...",markant flere,"antallet af danske sygeplejersker, der har fåe...",ritzau/,Penge,2012-01-19 13:27:00+00:00,https://www.dr.dk/nyheder/penge/danske-sygeple...,DR,"[mens, nyuddannede, danske, sygeplejersker, ha...","antal af dansk sygeplejersker, der har fået au..."
1,1,næsten ingen ledige sygeplejersker,trods fyringsrunder i både 2010 og 2011 er arb...,NaN,trods fyringsrunder på sygehusene i både 2010 ...,henny mortensen,Sjælland,2012-01-28 07:42:00+00:00,https://www.dr.dk/nyheder/regionale/sjaelland/...,DR,"[antallet, af, danske, sygeplejersker, der, ha...",trod fyringsrund på sygehus i båd 2010 og 2011...
2,5,regionsformand: jeg har ikke noget at undskylde,"steen bach nielsen forstår ikke, at sygeplejer...",NaN,der er ikke noget at undskylde eller beklage.....,jørgen hansen,Sjælland,2012-01-24 15:39:00+00:00,https://www.dr.dk/nyheder/regionale/sjaelland/...,DR,"[en, bevæbnet, mand, har, skudt, og, dræbt, sy...",der er ikk nog at undskyld ell beklage... form...
3,14,sygeplejersker vil skære i nattevagter,risiko for at natarbejde er kræftfremkaldende ...,færre nattevagter fast døgnrytme er vigtig,"dansk sygeplejeråd, der repræsenterer landets ...",ritzau,Indland,2012-02-21 10:48:00+00:00,https://www.dr.dk/nyheder/indland/sygeplejersk...,DR,"[hvert, år, dør, hundredvis, af, patienter, fo...","dansk sygeplejeråd, der repræsent land omkring..."
4,15,udenlandske sygeplejersker er taget hjem,det er slut med sygeplejerudtryk på gebrokkent...,NaN,det er slut med sygeplejerudtryk på gebrokkent...,mikkel from nielsen,Nordjylland,2012-02-27 15:26:00+00:00,https://www.dr.dk/nyheder/regionale/nordjyllan...,DR,"[sygeplejersker, lærere, politifolk, og, unge,...",det er slut med sygeplejerudtryk på gebrokkent...
...,...,...,...,...,...,...,...,...,...,...,...,...
523,1495,læger og sygeplejersker siger stop: 'kan ikke ...,"ifølge ansatte på holbæk sygehus, så er forhol...",besøg fra arbejdstilsynet klare krav,"det er ikke kun sygeplejersker, der flygter fr...",trine warrer juul,Sjælland,2021-12-06 04:55:00+00:00,https://www.dr.dk/nyheder/regionale/sjaelland/...,DR,"[sundhedsstyrelsen, kom, torsdag, med, hård, k...","det er ikk kun sygeplejersker, der flygt fra r..."
524,1496,detektor: talsperson erkender - har ikke tal p...,talsperson for opsigelser blandt sygeplejerske...,debatindlæg som dokumenation sundhedsøkonom: '...,"- patienter ligger på gangene og dør, fordi de...",august stenbroen,Detektor,2021-12-22 19:39:00+00:00,https://www.dr.dk/nyheder/detektor/detektor-ta...,DR,"[stop, det, offentlige, forbrug, og, sænk, ska...","- patient lig på gang og dør, fordi de ikk bli..."
525,1497,flere sygeplejersker skifter til det private: ...,der skal gøres noget ved løn og vilkår i det o...,fik hjertebanken sygeplejerskerne siger fra ’i...,"højere løn og bedre arbejdstider. det var det,...",allan nisgaard,Indland,2021-12-29 04:46:00+00:00,https://www.dr.dk/nyheder/indland/flere-sygepl...,DR,"[der, er, mange, forsinkelser, og, fejl, i, va...","høj løn og bedr arbejdstider. det var det, som..."
526,1498,nu skal sygeplejersker og læger fra hospitaler...,i både region midtjylland og region syddanmark...,frygter længere ventelister ansætter mange,meldingen fra sundhedsmyndighederne har været ...,emil eller,Indland,2021-12-15 04:50:00+00:00,https://www.dr.dk/nyheder/indland/nu-skal-syge...,DR,"[36, årige, kirsten, brosbøl, bliver, ny, milj...",melding fra sundhedsmyndighed har vær helt kla...


In [ ]:
    stemmed_tokens = [porter_stemmer.stem(token) for token in tokens]

In [59]:
count = CountVectorizer() #Store the class in 'count' to ease coding
count_array = dr_2['stems'] #Take the first two reviews and store them in an array
bag = count.fit_transform(count_array) #fit_

In [62]:
array = bag.toarray() #Make the bag to an array
matrix = pd.DataFrame(data=array,columns = count.get_feature_names_out()) #Input the bag and the words into a dataframe
matrix

,00,000,001,007,015,02,030,031,04,042,...,østjysk,østkyst,østr,øve,øvels,øvelse,øver,øverst,øvet,øvr
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
523,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
524,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
525,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
526,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [63]:
matrix_sum = matrix.sum().transpose()
matrix_sum.sort_values(ascending = False)

at                 8419
er                 6830
det                6463
og                 5902
der                4785
                   ... 
gastromedicinsk       1
gastromé              1
posttraumatisk        1
postkas               1
loyalit               1
Length: 12915, dtype: int64

In [45]:
for i in dr_2['tokenized']:
    dr_2['stemmed']=[stemmer.stem(word) for word in i]

ValueError: Length of values (503) does not match length of index (528)

In [ ]:
wordlist_stemmed_with_num = [stemmer.stem(row['tokenized']) for word in row['tokenized']]

In [5]:
# Define a tokenizer function
def tokenizer(df):
    df_tokenslist = []
    for document in tqdm.tqdm(df['content']):
        tokens = nltk.tokenize.word_tokenize(document, language = 'danish')
        df_tokenslist.append(tokens)
    df_tokens = list(itertools.chain(*df_tokenslist))
    return df_tokens

In [6]:
dr_2_tokens = tokenizer(dr_2)
print(len(dr_2_tokens))
tv2_2_tokens = tokenizer(tv2_2)
print(len(tv2_2_tokens))
ft_2_tokens = tokenizer(ft_2)
print(len(ft_2_tokens))

100%|██████████| 528/528 [00:00<00:00, 818.88it/s] 


232998


100%|██████████| 3607/3607 [00:06<00:00, 549.80it/s]


2562567


100%|██████████| 296/296 [00:16<00:00, 17.44it/s]

7843399


## Remove stopwords and create word dictionary

In [7]:
# Get stopwords list
nltk.download('stopwords')
stopwords = nltk.corpus.stopwords.words('danish')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/oliver/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


### Tokenized content for three datasets

In [8]:
# Remove stopwords from token-list
dr_nostop = [word for word in dr_2_tokens if not word in stopwords]
tv2_nostop = [word for word in tv2_2_tokens if not word in stopwords]
ft_nostop = [word for word in ft_2_tokens if not word in stopwords]

### Create set of unique words

In [9]:
wordlist_complete = dr_nostop + tv2_nostop + ft_nostop

In [10]:
len(wordlist_complete)

5356096

The total number of words across our three datasets is 5356096.

Our unique wordset contains 115189 words.

## Stemming

In [38]:
stemmer = SnowballStemmer("danish")

In [12]:
[stemmer.stem(word) for word in ['tokenized']]

In [13]:
# Delete duplicates
wordlist_stemmed_with_num = list(set(wordlist_stemmed_with_num))
len(wordlist_stemmed_with_num)

75802

In [14]:
# Remove numbers
wordlist_stemmed = [word for word in wordlist_stemmed_with_num if not word.isdigit()]            

## Lemmatization

In [28]:
# Load Danish lemmatizer
lem = lemmy.load("da")

In [166]:
wordlist_lem = [lem.lemmatize("", word) for word in wordset]

In [ ]:
# Create a list instead of list of list
wordlist_lem = [word for sublist in wordlist_lem for word in sublist]

In [164]:
wordlist_lem_2 = wordlist_lem_2 

list

Comment: The lemmatization returns a list of lists that also contains more than two words which could lead to problems.
    

## Bag of words

In [32]:
dr_matrix_sum = pd.read_csv('dr_matix_sum.csv').set_index('0')

In [33]:
dr_matrix_sum.rename(

,Unnamed: 0
0,
2,00
273,000
1,001
1,007
1,015
...,...
2,øvels
4,øver
10,øverst
